In [103]:
import numpy as np
import pandas as pd
from sklearn.model_selection import train_test_split
from sklearn.preprocessing import StandardScaler, LabelEncoder
from sklearn.utils.class_weight import compute_class_weight
from tensorflow.keras.models import Sequential
from tensorflow.keras.layers import Dense

In [85]:
data = pd.read_csv('seattle-weather.csv')

In [86]:
data['date'] = pd.to_datetime(data['date'], format='%Y-%m-%d')

In [87]:
data

,date,precipitation,temp_max,temp_min,wind,weather
0,2012-01-01,0.0,12.8,5.0,4.7,drizzle
1,2012-01-02,10.9,10.6,2.8,4.5,rain
2,2012-01-03,0.8,11.7,7.2,2.3,rain
3,2012-01-04,20.3,12.2,5.6,4.7,rain
4,2012-01-05,1.3,8.9,2.8,6.1,rain
...,...,...,...,...,...,...
1456,2015-12-27,8.6,4.4,1.7,2.9,rain
1457,2015-12-28,1.5,5.0,1.7,1.3,rain
1458,2015-12-29,0.0,7.2,0.6,2.6,fog
1459,2015-12-30,0.0,5.6,-1.0,3.4,sun


In [88]:
lab_encode = LabelEncoder()

In [89]:
data['weather'] = lab_encode.fit_transform(data['weather'])

In [90]:
X = data[['precipitation', 'temp_max', 'temp_min', 'wind']]
Y = data['weather']

In [91]:
X_train, X_test, Y_train, Y_test = train_test_split(X, Y, test_size = 0.2, random_state = 42)

In [92]:
scale = StandardScaler()
X_train_scaled = scale.fit_transform(X_train)
X_test_scaled = scale.transform(X_test)

In [93]:
class_weights = compute_class_weight('balanced', classes=np.unique(Y_train), y=Y_train)
class_weight = dict(zip(np.unique(Y_train), class_weights))

In [94]:
model = Sequential()
model.add(Dense(128, activation='relu', input_shape=(X_train_scaled.shape[1],)))
model.add(Dense(64, activation='relu'))
model.add(Dense(len(np.unique(Y)), activation='softmax')) 

In [95]:
model.compile(optimizer = 'Adam', loss = 'sparse_categorical_crossentropy', metrics = ['accuracy'])

In [100]:

model.fit(X_train_scaled, Y_train, epochs=1000, batch_size=32, validation_split=0.2, class_weight=class_weight)

Epoch 1/1000
30/30 [==============================] - 0s 13ms/step - loss: 0.5791 - accuracy: 0.7323 - val_loss: 0.8266 - val_accuracy: 0.6496
Epoch 2/1000
30/30 [==============================] - 0s 10ms/step - loss: 0.5743 - accuracy: 0.7291 - val_loss: 0.8396 - val_accuracy: 0.6538
Epoch 3/1000
30/30 [==============================] - 0s 9ms/step - loss: 0.5738 - accuracy: 0.7248 - val_loss: 0.7833 - val_accuracy: 0.6966
Epoch 4/1000
30/30 [==============================] - 0s 10ms/step - loss: 0.5731 - accuracy: 0.7495 - val_loss: 0.8444 - val_accuracy: 0.6197
Epoch 5/1000
30/30 [==============================] - 0s 10ms/step - loss: 0.5958 - accuracy: 0.6767 - val_loss: 0.8038 - val_accuracy: 0.6923
Epoch 6/1000
30/30 [==============================] - 0s 10ms/step - loss: 0.5705 - accuracy: 0.7741 - val_loss: 0.8798 - val_accuracy: 0.6410
Epoch 7/1000
30/30 [==============================] - 0s 10ms/step - loss: 0.5963 - accuracy: 0.7248 - val_loss: 0.8660 - val_accuracy: 0.6538


In [104]:
loss, accuracy = model.evaluate(X_test_scaled, Y_test)
print(f'Test Accuracy: {accuracy}')

10/10 [==============================] - 0s 8ms/step - loss: 1.3133 - accuracy: 0.7850
Test Accuracy: 0.7849829196929932


In [117]:
custom_input = np.array([0, 12.8, 5.0, 4.3])
custom_input_scaled = scale.transform(custom_input.reshape(1, -1))
predictions = model.predict(custom_input_scaled)
predicted_class = lab_encode.classes_[np.argmax(predictions)]
print(f'Predicted Weather: {predicted_class}')

1/1 [==============================] - 0s 34ms/step
Predicted Weather: drizzle


/root/anaconda3/lib/python3.11/site-packages/sklearn/base.py:464: UserWarning: X does not have valid feature names, but StandardScaler was fitted with feature names
  warnings.warn(
